In [1]:
import pandas as pd
import numpy as np
import csv

### Data gathered from:

- California WARN data: https://edd.ca.gov/Jobs_and_Training/Layoff_Services_WARN.htm
- population data from: https://www.census.gov/data/tables/time-series/demo/popest/2010s-counties-total.html#par_textimage_242301767

### This notebook will explore layoffs in counties with high rent burdens

In the [finding_ambiguous_cities](https://github.com/biglocalnews/WARN_Cali_analysis/blob/master/hardest_hit/finding_ambiguous_cities.ipynb) notebook, we've already standardized the county information. We've also identified a few records with the incorrect city name, so we need to clean those records. 

In [2]:
cali_layoffs = pd.read_csv('california_warn_raw_2.csv')
cali_population = pd.read_csv('county_population.csv', delimiter='\t', thousands=',')

In [3]:
cali_layoffs.head()

,Unnamed: 0,Notice Date,Effective Date,Received Date,Company,City,County,Employees,Layoff/Closure,County Orig
0,0,10/26/2020,12/28/2020,11/10/2020,DAMAC Products LLC,La Mirada,Los Angeles County,79.0,Closure Permanent,Los Angeles County
1,1,10/27/2020,12/31/2020,11/10/2020,Gulfstream,Long Beach,Los Angeles County,608.0,Closure Permanent,Los Angeles County
2,2,10/26/2020,12/16/2020,11/10/2020,Renaissance Newport Beach Hotel,Newport Beach,Orange County,104.0,Closure Permanent,Orange County
3,3,10/01/2020,10/05/2020,11/10/2020,Howmet Aerospace,Rancho Cucamonga,San Bernardino County,66.0,Layoff Temporary,San Bernardino County
4,4,10/26/2020,07/07/2020,11/10/2020,"Newport Diversified, Inc.",El Cajon,San Diego County,91.0,Layoff Temporary,San Diego County


In [4]:
cali_layoffs.tail()

,Unnamed: 0,Notice Date,Effective Date,Received Date,Company,City,County,Employees,Layoff/Closure,County Orig
9502,9502,06/25/2015,08/24/2015,06/30/2015,"Intuit, Inc.",San Francisco,No Name,86.0,Layoff Permanent,No County Name
9503,9503,06/25/2015,08/24/2015,06/30/2015,"Intuit, Inc.",Santa Monica,No Name,49.0,Closure Permanent,No County Name
9504,9504,06/25/2015,08/24/2015,06/30/2015,"Intuit, Inc.",Venice,No Name,11.0,Closure Permanent,No County Name
9505,9505,06/29/2015,08/28/2015,06/30/2015,"Safeway, Inc.",Pleasanton,No Name,18.0,Layoff Unknown at this time,No County Name
9506,9506,06/30/2015,07/22/2015,06/30/2015,Medtronic Ablation Frontiers LLC,Carlsbad,No Name,50.0,Closure Permanent,No County Name


### California Layoffs

For the purposes of this analysis, we are selecting 2020 records by isolating the year on the `Notice date` column.
By choosing `Notice date` we make sure that these layoffs were issued after the start of shelter in place for 2020, as some layoff notices can be issues much earlier the layoff date.

In [5]:
cali_layoffs['Year'] = cali_layoffs['Notice Date'].str[-4:]
cali_layoffs.head()

,Unnamed: 0,Notice Date,Effective Date,Received Date,Company,City,County,Employees,Layoff/Closure,County Orig,Year
0,0,10/26/2020,12/28/2020,11/10/2020,DAMAC Products LLC,La Mirada,Los Angeles County,79.0,Closure Permanent,Los Angeles County,2020
1,1,10/27/2020,12/31/2020,11/10/2020,Gulfstream,Long Beach,Los Angeles County,608.0,Closure Permanent,Los Angeles County,2020
2,2,10/26/2020,12/16/2020,11/10/2020,Renaissance Newport Beach Hotel,Newport Beach,Orange County,104.0,Closure Permanent,Orange County,2020
3,3,10/01/2020,10/05/2020,11/10/2020,Howmet Aerospace,Rancho Cucamonga,San Bernardino County,66.0,Layoff Temporary,San Bernardino County,2020
4,4,10/26/2020,07/07/2020,11/10/2020,"Newport Diversified, Inc.",El Cajon,San Diego County,91.0,Layoff Temporary,San Diego County,2020


### Cleaning Layoff Types

Type Unknown and unknown at this time will be combined.

In [6]:
cali_layoffs['Layoff/Closure'].unique()

array(['Closure Permanent', 'Layoff Temporary', 'Closure Temporary',
       'Layoff Permanent', 'Layoff Type Unknown', 'Closure Type Unknown',
       'Closure Unknown at thistime', 'Layoff Unknown at thistime',
       'Layoff Unknown at this time', 'Closure Unknown at this time'],
      dtype=object)

In [7]:
cali_layoffs['Layoff/Closure clean'] = cali_layoffs['Layoff/Closure'].str.replace('at thistime', 'at this time')
cali_layoffs['Layoff/Closure clean'] = cali_layoffs['Layoff/Closure clean'].str.strip()
cali_layoffs['Layoff/Closure clean'] = cali_layoffs['Layoff/Closure clean'].str.lower()
cali_layoffs['Layoff/Closure clean'] = cali_layoffs['Layoff/Closure clean'].str.replace('unknown at this time', 'type unknown')
cali_layoffs['Layoff/Closure clean'].unique()

array(['closure permanent', 'layoff temporary', 'closure temporary',
       'layoff permanent', 'layoff type unknown', 'closure type unknown'],
      dtype=object)

In [8]:
years = ['2020', '2019']

### Extracting Only Necessary Records

Now that the data has been standardized, we will be selecting the records that have a notice date from the year 2020 or 2019. This will allow for comparisons between a pandemic and non-pandemic year in terms of layoff notices. 

In [9]:
cali_2020_2019 = cali_layoffs[cali_layoffs["Year"].isin(years)]
cali_2020_2019.head()

,Unnamed: 0,Notice Date,Effective Date,Received Date,Company,City,County,Employees,Layoff/Closure,County Orig,Year,Layoff/Closure clean
0,0,10/26/2020,12/28/2020,11/10/2020,DAMAC Products LLC,La Mirada,Los Angeles County,79.0,Closure Permanent,Los Angeles County,2020,closure permanent
1,1,10/27/2020,12/31/2020,11/10/2020,Gulfstream,Long Beach,Los Angeles County,608.0,Closure Permanent,Los Angeles County,2020,closure permanent
2,2,10/26/2020,12/16/2020,11/10/2020,Renaissance Newport Beach Hotel,Newport Beach,Orange County,104.0,Closure Permanent,Orange County,2020,closure permanent
3,3,10/01/2020,10/05/2020,11/10/2020,Howmet Aerospace,Rancho Cucamonga,San Bernardino County,66.0,Layoff Temporary,San Bernardino County,2020,layoff temporary
4,4,10/26/2020,07/07/2020,11/10/2020,"Newport Diversified, Inc.",El Cajon,San Diego County,91.0,Layoff Temporary,San Diego County,2020,layoff temporary


In [10]:
pd.options.display.max_rows = 999
cali_2020_2019[cali_2020_2019['City'] == 'Sacramento']

,Unnamed: 0,Notice Date,Effective Date,Received Date,Company,City,County,Employees,Layoff/Closure,County Orig,Year,Layoff/Closure clean
10,10,10/06/2020,11/05/2020,11/09/2020,Natomas Sports Club,Sacramento,Sacramento County,89.0,Closure Permanent,Sacramento County,2020,closure permanent
11,11,09/30/2020,09/30/2020,11/09/2020,"WASH Multifamily Laundry Systems, LLC",Sacramento,Sacramento County,1.0,Layoff Permanent,Sacramento County,2020,layoff permanent
23,23,07/23/2020,09/22/2020,11/05/2020,"Crescent Hotels and Resorts, LLC",Sacramento,Sacramento County,30.0,Layoff Permanent,Sacramento County,2020,layoff permanent
53,53,08/20/2020,08/08/2020,10/28/2020,"Aldo US Inc., Store 2077",Sacramento,Sacramento County,10.0,Layoff Temporary,Sacramento County,2020,layoff temporary
128,128,09/01/2020,05/03/2020,10/20/2020,"Taste, Inc.",Sacramento,Sacramento County,8.0,Layoff Temporary,Sacramento County,2020,layoff temporary
158,158,10/02/2020,03/20/2020,10/09/2020,Hyatt Corporation dba Hyatt Regency Sacramento,Sacramento,Sacramento County,242.0,Layoff Permanent,Sacramento County,2020,layoff permanent
304,304,09/03/2020,11/02/2020,09/28/2020,Liqui-Box Corporation,Sacramento,Sacramento County,22.0,Layoff Permanent,Sacramento County,2020,layoff permanent
378,378,09/24/2020,12/27/2020,09/25/2020,"McClatchy Company, LLC",Sacramento,Sacramento County,199.0,Layoff Temporary,Sacramento County,2020,layoff temporary
388,388,09/15/2020,11/02/2020,09/24/2020,Liquibox,Sacramento,Sacramento County,22.0,Closure Permanent,Sacramento County,2020,closure permanent
666,666,03/20/2020,03/20/2020,09/03/2020,"Paradies Shops, LLC",Sacramento,Sacramento County,37.0,Layoff Temporary,Sacramento County,2020,layoff temporary


### More Data Cleaning

A few records have the incorrect county, based on the information in the city column. These cities include: Sacramento, Hayward, Indian Wells, Cerritos, Vista, Roseville, City of Industry, Del Mar, Walnut Creek, and Los Gatos. For information on how these cities were selected, see the [finding_ambiguous_cities](https://github.com/biglocalnews/WARN_Cali_analysis/blob/master/hardest_hit/finding_ambiguous_cities.ipynb) notebook.

In [11]:
dict_cities = {
    'Sacramento':'Sacramento County',
    'San Francisco':'San Francisco County',
    'Hayward':'Alameda County',
    'Indian Wells':'Riverside County',
    'Cerritos':'Los Angeles County',
    'Vista':'San Diego County',
    'Roseville':'Placer County',
    'City of Industry':'Los Angeles County',
    'Del Mar':'San Diego County',
    'Walnut Creek':'Contra Costa County',
    'Los Gatos':'Santa Clara County'
}

In [16]:
### IMPLEMENT CODE TO HAVE CORRECT COUNTY NAMES FOR CITIES
for city, county in dict_cities.items():
    cali_2020_2019.loc[cali_2020_2019['City'] == city, 'County'] = county
    

Sacramento Sacramento County
San Francisco San Francisco County
Hayward Alameda County
Indian Wells Riverside County
Cerritos Los Angeles County
Vista San Diego County
Roseville Placer County
City of Industry Los Angeles County
Del Mar San Diego County
Walnut Creek Contra Costa County
Los Gatos Santa Clara County


In [17]:
# pd.options.display.max_rows = 999

# cali_2020_2019['County'] = cali_2020_2019['County'].fillna(cali_2020_2019['City'].astype(str).map(dict_cities))
# cali_2020_2019.head()
# # cali_2020_2019['City'] = cali_2020_2019['City'].fillna(cali_2020_2019['County'].astype(str).map(dict_cities))
# cali_2020_2019[cali_2020_2019['City'] == 'Sacramento']

In [14]:
### Export File to clean company names in open refine.
### Re-import clean company names file.

### Adding Population Data

The data has been grabbed from the census link above. These figures will be used to calculate per capita layoff numbers per county.

In [15]:
cali_population.rename(columns={'California':'Counties','39,512,223':'Population'}, inplace=True)
cali_population['County Names'] = cali_population['Counties'].str[1:]
cali_population['County Names'] = cali_population['County Names'].str.split(',').str[0]
cali_population.head(4)

,Counties,Population,County Names
0,".Alameda County, California",1671329,Alameda County
1,".Alpine County, California",1129,Alpine County
2,".Amador County, California",39752,Amador County
3,".Butte County, California",219186,Butte County
